# lib

In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=2bd29ab9aefc04305eb63c4244479a7d9f901b2198f51e3a4bacf49468603fd2
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
Note: you may need to restart the kernel to use updated packages.


In [3]:
!mkdir -p vncorenlp/models/wordsegmenter

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar

!mv VnCoreNLP-1.1.1.jar vncorenlp/



!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr

!mv vi-vocab vncorenlp/models/wordsegmenter/

!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/





!mkdir -p vncorenlp/models/postagger

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/postagger/vi-tagger 

!mv vi-tagger vncorenlp/models/postagger/





!mkdir -p vncorenlp/models/ner

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-500brownclusters.xz

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-ner.xz 

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-pretrainedembeddings.xz

!mv vi-500brownclusters.xz vncorenlp/models/ner/

!mv vi-ner.xz vncorenlp/models/ner/

!mv vi-pretrainedembeddings.xz vncorenlp/models/ner/

--2024-12-15 12:32:26--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: 'VnCoreNLP-1.1.1.jar'

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.09s   

2024-12-15 12:32:27 (301 MB/s) - 'VnCoreNLP-1.1.1.jar' saved [27412575/27412575]

--2024-12-15 12:32:29--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting respon

In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keras-preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
VNCORE_NLP = False

max_len = 50

In [7]:
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg") 

In [8]:
def tokennize_vn(text):

    sentences = vncorenlp.tokenize(text)

    s = ''

    for t in sentences:

        s = s + ' '.join(t) + ' '

    return s

# Read data

In [9]:
train_dataset = '/kaggle/input/vithsd-norm-aug/Train_normalize_aug.csv'

dev_dataset = '/kaggle/input/vithsd-norm-aug/Dev_normalize.csv'

test_dataset = '/kaggle/input/vithsd-norm-aug/Test_normalize.csv'

In [10]:
import pandas as pd



train = pd.read_csv(train_dataset)

dev = pd.read_csv(dev_dataset)

test = pd.read_csv(test_dataset)

In [11]:
len(train)

12984

In [12]:
len(dev)

1201

In [13]:
len(test)

1800

# Feature extraction  

## Make data

In [14]:
import itertools



aspect = list(train.columns.values[2:])

sent = [1,2,3]



itertools.product([aspect, sent])



combined_label = [a for a in itertools.product(aspect, sent)]

In [15]:
combined_label

[('individual', 1),
 ('individual', 2),
 ('individual', 3),
 ('groups', 1),
 ('groups', 2),
 ('groups', 3),
 ('religion/creed', 1),
 ('religion/creed', 2),
 ('religion/creed', 3),
 ('race/ethnicity', 1),
 ('race/ethnicity', 2),
 ('race/ethnicity', 3),
 ('politics', 1),
 ('politics', 2),
 ('politics', 3)]

In [16]:
import numpy as np 

import pandas as pd



def make_data(data):

    content = []

    label = []



    for i in range(0, len(data)):

        row = data.iloc[i]

        if VNCORE_NLP:

            content.append(tokennize_vn(str(row['content'])))

        else:

            content.append(row['content'])

        default_lab = np.zeros(len(combined_label), dtype=int)

        for j in range(0, len(combined_label)):

            l = combined_label[j]

            if row[l[0]] == l[1]:

                default_lab[j] = 1

        label.append(default_lab)



    return pd.DataFrame({

        'Content': content,

        'Labels': label

    })

In [17]:
train_data = make_data(train)

dev_data = make_data(dev)

test_data = make_data(test)

In [18]:
train_data['Content'] = train_data['Content'].astype(str)
dev_data['Content'] = dev_data['Content'].astype(str)
test_data['Content'] = test_data['Content'].astype(str)


In [19]:
test_data[1:10]

,Content,Labels
1,đúng câu lé sai 🤔🤔🤔 khẩu nghiệp già tục tĩu dạ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"đéo 1 thằng đui não trưởng , : ) )","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,hiếu bùi sủa dơ,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,anh trăm nayd cố găng lỗ đít mọc đinh 🤧🤧 kiểu ...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,"thái lan hèn thật , ta đội ngũ tiên phong giai...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
6,"chau bao nhiêu la , cua , hả","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,sao đoạn ta,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,"về trị , tuyệt đối nhân dân ta chút dân chủ nào .","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
9,gọi mấy thằng công an là phải dí thẳng ...,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Features

In [20]:
train_data['Content']

0        chào báo vũ hoàng lân . chào danh hài vũ trụ t...
1                              anh thi đi đăng ký rủ nhé .
2                                                   mỹ lồn
3          thôn thôn lầm , thôn lầm dân ghét , bật thôn lù
4        phạm văn lộc phụ hồ trời nắng chóng mặt hả thầ...
                               ...                        
12979    lê luân sánh lương . mà sánh quyền lực . làm c...
12980    lê luân đừng sánh lương . mà sánh ích . làm cô...
12981    lê luân đừng sánh lương mà sánh quyền lực ích ...
12982    lê luân đừng lương . mà sánh quyền lực ích làm...
12983    lê luân đừng sánh lương . mà sánh quyền lực íc...
Name: Content, Length: 12984, dtype: object

In [21]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.text import Tokenizer



tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_data['Content'])

vocab_size = len(tokenizer.word_index) + 1



def encoding(X, y):

    X = tokenizer.texts_to_sequences(X.values.astype(str))

    X = pad_sequences(X, maxlen=max_len, padding='post')

    y = np.asarray([np.asarray(row, dtype=float) for row in y], dtype=int)

    return (X,y)

In [22]:
# import pickle



# # saving

# with open('drive/MyDrive/CODE/ViTHSD/model/tokenizer1.pickle', 'wb') as handle:

#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
train_features = encoding(train_data['Content'], train_data['Labels'])

dev_features = encoding(dev_data['Content'], dev_data['Labels'])

test_features = encoding(test_data['Content'], test_data['Labels'])

## Evaluation metric

In [24]:
# Evaluation metric

import sys

import os

import os.path

from scipy.stats import sem

import numpy as np

from ast import literal_eval

import tensorflow as tf



def em_score(y_true, y_pred):

    MR = np.all(y_pred == y_true, axis=1).mean()

    return MR



def accuracy_score(y_true, y_pred):

    temp = 0

    for i in range(0, len(y_true)):

        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    
    return temp / len(y_true)


def f1_score(y_true, y_pred):

    temp = 0

    for i in range(len(y_true)):

        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):

            continue

        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i]))) / (sum(y_true[i])+sum(y_pred[i]))

    return temp/ len(y_true)

def recall(y_true, y_pred):
    temp = 0

    for i in range(len(y_pred)):
        if(sum(y_true[i]) == 0) and (sum(y_pred[i])==0):
            continue
        if sum(y_true[i]) == 0:
            continue
        temp += sum(np.logical_and(y_true[i],y_pred[i])) / sum(y_true[i])
    return temp / len(y_pred)

def precision(y_true, y_pred):
    temp = 0

    for i in range(len(y_pred)):
        if(sum(y_true[i]) == 0) and (sum(y_pred[i])==0):
            continue
        if sum(y_pred[i]) ==0:
            continue
        temp += sum(np.logical_and(y_true[i],y_pred[i])) / sum(y_pred[i])
    return temp / len(y_pred)

# Model

## Pre-trained Embedding 

In [25]:
import numpy as np



EMBEDING_DIM = 300

NUM_LABEL = 15

EMBEDDING = '/kaggle/input/vi-300-vec/cc.vi.300.vec'

MAX_FEATURE = 10000



embeddings_index = dict()

f = open(EMBEDDING)

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index[word] = coefs

f.close()

print('Loaded %s word vectors.' % len(embeddings_index))





embedding_matrix = np.zeros((vocab_size, EMBEDING_DIM))

for word, i in tokenizer.word_index.items():

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        embedding_matrix[i] = embedding_vector

Loaded 2000000 word vectors.


## Train 

In [26]:
# Bi-GRU-LSTM-CNN



import tensorflow as tf

from tensorflow import keras



from keras.models import Sequential, Model



from keras.layers import Dense, Concatenate, Bidirectional, GRU, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Dropout, concatenate

from keras.layers import Flatten, LSTM, Input

from tensorflow.keras.layers import Embedding



units = 100



input = Input(shape = (max_len,))

emb = Embedding(vocab_size, EMBEDING_DIM, weights=[embedding_matrix], input_length=max_len, trainable=True)(input)

x1 = SpatialDropout1D(0.2)(emb)



x = Bidirectional(GRU(units, return_sequences = True))(x1)

x = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)

    

y = Bidirectional(LSTM(units, return_sequences = True))(x1)

y = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)

    

avg_pool1 = GlobalAveragePooling1D()(x)

max_pool1 = GlobalMaxPooling1D()(x)

    

avg_pool2 = GlobalAveragePooling1D()(y)

max_pool2 = GlobalMaxPooling1D()(y)

    

    

x = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])

out = Dense(15, activation = "sigmoid")(x)



model = Model(input, out)



model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 300)   │  1,915,500 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d   │ (None, 50, 300)   │          0 │ embedding[0][0]   │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 50, 200)   │    241,200 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 50, 200)   │    320,800 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 49, 50)    │     20,050 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 49, 50)    │     20,050 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 50)        │          0 │ conv1d[0][0]      │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 50)        │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 50)        │          0 │ conv1d_1[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 50)        │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 200)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 15)        │      3,015 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,520,615 (9.62 MB)

 Trainable params: 2,520,615 (9.62 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.fit(train_features[0].astype(np.float32), train_features[1], validation_data=(dev_features[0].astype(np.float32), dev_features[1]), 

          batch_size=128, epochs=30, verbose=True)

Epoch 1/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - acc: 0.2941 - loss: 0.3011 - val_acc: 0.5387 - val_loss: 0.2106
Epoch 2/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.7275 - loss: 0.1405 - val_acc: 0.5612 - val_loss: 0.2075
Epoch 3/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.8035 - loss: 0.0987 - val_acc: 0.6028 - val_loss: 0.1984
Epoch 4/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.8262 - loss: 0.0797 - val_acc: 0.5504 - val_loss: 0.2069
Epoch 5/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.8422 - loss: 0.0680 - val_acc: 0.5362 - val_loss: 0.2179
Epoch 6/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.8519 - loss: 0.0569 - val_acc: 0.5745 - val_loss: 0.2273
Epoch 7/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - acc: 0.8617 - loss: 0.0480 - val_acc: 0.5420 - val_loss: 0.2515
Epoch 8/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - acc: 0.8638 - loss: 0.0406 - val_acc: 0.5321 - val_loss: 0.2728
Epoch 9/30
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms

## Eval

In [28]:
y_test_pred = model.predict(test_features[0])

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [30]:
y_test_pred_new = []



for y in y_test_pred:

    lb = []

    for i in range(0, len(y)):

        if y[i] >= 0.5:

            lb.append(1)

        else:

            lb.append(0)

    y_test_pred_new.append(lb)

#Aspect
new_pred = []

new_y = []

for k in range(len(y_test_pred_new)):
    result = np.sum(np.array(y_test_pred_new[k]).reshape(-1,3),axis = 1)
    new_pred.append(result)


for k in range(len(test_features[1])):
    result = np.sum(test_features[1][k].reshape(-1,3),axis = 1)
    new_y.append(result)

In [31]:
print("Aspect + Polarity\n")

print("F1 = {}".format(f1_score(test_features[1], y_test_pred_new) * 100))

print("Precision = {}".format(precision(test_features[1], y_test_pred_new)* 100))

print("Recall = {}".format(recall(test_features[1], y_test_pred_new)* 100))

Aspect + Polarity

F1 = 37.09775132275131
Precision = 38.812037037037044
Recall = 38.44444444444445


In [32]:
print("Aspect\n")

print("F1 = {}".format(f1_score(new_y, new_pred) * 100))

print("Precision = {}".format(precision(new_y, new_pred) * 100))

print("Recall = {}".format(recall(new_y, new_pred) * 100))

Aspect

F1 = 53.064417989417656
Precision = 56.12685185185183
Recall = 54.78703703703706
